In [0]:
!pip install tensorflow-gpu


     |████████████████████████████████| 421.8MB 41kB/s 
     |████████████████████████████████| 450kB 53.8MB/s 
     |████████████████████████████████| 3.9MB 47.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=b10b3be589c3982771c059b2839944b2e6d14ec19d901e4ca530d7b697234825
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0rc2 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc2 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 2.2.0rc2 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Fo

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

import tensorflow_datasets as tfds

import os
import re
import numpy as np

import matplotlib.pyplot as plt


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
questions=[]

for file in sorted(os.listdir("drive/My Drive/sparknotes/merged_except_romeo")):
    if file.endswith("_modern.snt.aligned"):
        filename_concat='drive/My Drive/sparknotes/merged_except_romeo/'+file
        with open(filename_concat) as f:
            lines_modern = f.read().splitlines()

        questions=questions+lines_modern


for file in sorted(os.listdir("drive/My Drive/enotes/merged")):
    if file.endswith("_modern.snt.aligned"):
        filename_concat='drive/My Drive/enotes/merged/'+file
        with open(filename_concat) as f:
            lines_modern = f.read().splitlines()

        questions=questions+lines_modern


In [0]:
answers=[]

for file in sorted(os.listdir("drive/My Drive/sparknotes/merged_except_romeo")):
    if file.endswith("_original.snt.aligned"):
        filename_concat='drive/My Drive/sparknotes/merged_except_romeo/'+file
        with open(filename_concat) as f:
            lines_modern = f.read().splitlines()

        answers=answers+lines_modern


for file in sorted(os.listdir("drive/My Drive/enotes/merged")):
    if file.endswith("_original.snt.aligned"):
        filename_concat='drive/My Drive/enotes/merged/'+file
        with open(filename_concat) as f:
            lines_modern = f.read().splitlines()

        answers=answers+lines_modern


In [0]:
len(questions)

29973

In [0]:
len(answers)

29973

In [0]:
questions[25098]

"You drive me past the bounds Of a maiden's patience."

In [0]:
answers[25098]

"Thou drivest me past the bounds Of maiden's patience."

In [0]:
questions[18000],answers[18000]

('Where’s the Master?', 'Where’s the Master?')

**Load and preprocess data**

To keep this example simple and fast, we are limiting the maximum number of training samples toMAX_SAMPLES=25000 and the maximum length of the sentence to be MAX_LENGTH=40.
We preprocess our dataset in the following order:

1. Extract MAX_SAMPLES conversation pairs into list of questions and answers.

2. Preprocess each sentence by removing special characters in each sentence.

3. Build tokenizer (map text to ID and ID to text) using TensorFlow Datasets SubwordTextEncoder.

4. Tokenize each sentence and add START_TOKEN and END_TOKEN to indicate the start and end of each sentence.

5. Filter out sentence that has more than MAX_LENGTH tokens.

6. Pad tokenized sentences to MAX_LENGTH


In [0]:
# Maximum number of samples to preprocess
#MAX_SAMPLES = 50000

def preprocess_sentence(sentence):
  sentence = sentence.lower().strip()
  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = re.sub(r'[" "]+', " ", sentence)
  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
  sentence = sentence.strip()
  # adding a start and an end token to the sentence
  return sentence


for i in range(len(questions)):
    questions[i]=preprocess_sentence(questions[i])
    
for i in range(len(answers)):
    answers[i]=preprocess_sentence(answers[i])

In [0]:
# Build tokenizer using tfds for both questions and answers
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13)

# Define start and end token to indicate the start and end of a sentence
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

# Vocabulary size plus start and end token
VOCAB_SIZE = tokenizer.vocab_size + 2

In [0]:
VOCAB_SIZE

8038

In [0]:
tokenizer.vocab_size

8036

In [0]:
tokenizer.encode(questions[122])

[289, 34, 1, 12, 1630, 8]

In [0]:
# Maximum sentence length
MAX_LENGTH = 40


# Tokenize, filter and pad sentences
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []
  
  for (sentence1, sentence2) in zip(inputs, outputs):
    # tokenize sentence
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN
    # check tokenized sentence max length
    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
      tokenized_inputs.append(sentence1)
      tokenized_outputs.append(sentence2)
  
  # pad tokenized sentences
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_inputs, tokenized_outputs


questions, answers = tokenize_and_filter(questions, answers)

In [0]:
print('Vocab size: {}'.format(VOCAB_SIZE))
print('Number of samples: {}'.format(len(questions)))

Vocab size: 8038
Number of samples: 28545


In [0]:
type(questions)

numpy.ndarray

In [0]:
questions.shape

(28545, 40)

In [0]:
answers.shape

(28545, 40)

In [0]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# decoder inputs use the previous target as input
# remove START_TOKEN from targets
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [0]:
dataset

<PrefetchDataset shapes: ({inputs: (None, 40), dec_inputs: (None, 39)}, {outputs: (None, 39)}), types: ({inputs: tf.int32, dec_inputs: tf.int32}, {outputs: tf.int32})>

In [0]:
def scaled_dot_product_attention(query, key, value, mask):
  """Calculate the attention weights. """
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # scale matmul_qk
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # add the mask to zero out padding tokens
  if mask is not None:
    logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k)
  attention_weights = tf.nn.softmax(logits, axis=-1)

  output = tf.matmul(attention_weights, value)

  return output

In [0]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # linear layers
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # split heads
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # scaled dot-product attention
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # concatenation of heads
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # final linear layer
    outputs = self.dense(concat_attention)

    return outputs

In [0]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [0]:
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

In [0]:
class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
    # apply sin to even index in the array
    sines = tf.math.sin(angle_rads[:, 0::2])
    # apply cos to odd index in the array
    cosines = tf.math.cos(angle_rads[:, 1::2])

    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [0]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [0]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [0]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [0]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [0]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)
  # mask the future tokens for decoder inputs at the 1st attention block
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
  # mask the encoder outputs for the 2nd attention block
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, enc_padding_mask])

  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [0]:
tf.keras.backend.clear_session()

# Hyper-parameters
NUM_LAYERS = 4
D_MODEL = 256
NUM_HEADS = 8
UNITS = 512
DROPOUT = 0.1

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

In [0]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

In [0]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [0]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # ensure labels have shape (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [0]:
EPOCHS = 20

model.fit(dataset, epochs=EPOCHS)

model.save_weights('drive/My Drive/dl/normaltrans_shakes_without_romeo20.h5')

Train for 447 steps
Epoch 1/20
447/447 [==============================] - 83s 186ms/step - loss: 2.3413 - accuracy: 0.0399
Epoch 2/20
447/447 [==============================] - 73s 163ms/step - loss: 1.6937 - accuracy: 0.0817
Epoch 3/20
447/447 [==============================] - 74s 165ms/step - loss: 1.4865 - accuracy: 0.0986
Epoch 4/20
447/447 [==============================] - 74s 165ms/step - loss: 1.3742 - accuracy: 0.1089
Epoch 5/20
447/447 [==============================] - 73s 163ms/step - loss: 1.2529 - accuracy: 0.1224
Epoch 6/20
447/447 [==============================] - 73s 163ms/step - loss: 1.1475 - accuracy: 0.1350
Epoch 7/20
447/447 [==============================] - 73s 162ms/step - loss: 1.0573 - accuracy: 0.1461
Epoch 8/20
447/447 [==============================] - 73s 163ms/step - loss: 0.9781 - accuracy: 0.1547
Epoch 9/20
447/447 [==============================] - 73s 163ms/step - loss: 0.9078 - accuracy: 0.1623
Epoch 10/20
447/447 [==============================] 

In [0]:
def evaluate(sentence):
  sentence = preprocess_sentence(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    # select the last word from the seq_len dimension
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # return the result if the predicted_id is equal to the end token
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # concatenated the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)


def predict(sentence):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

In [0]:
output = predict('Why, Romeo, are you crazy?')

Input: Why, Romeo, are you crazy?
Output: why , romeo , are you mad ?


In [0]:
output = predict('have you killed Tybalt?')

Input: have you killed Tybalt?
Output: hast thou slain tybalt ?


In [0]:
output = predict('bring her out to me .')

Input: bring her out to me .
Output: bring her before us .


In [0]:
output = predict('i will hit you !')

Input: i will hit you !
Output: i ll not .


In [0]:
output = predict('You are bad')

Input: You are bad
Output: thou rt kind .


In [0]:
output = predict('What is wrong with you ? ')

Input: What is wrong with you ? 
Output: what is your passion ?


In [0]:
output = predict('Don’t worry about me .')

Input: Don’t worry about me .
Output: fear me .


In [0]:
output = predict('Look what you made me do .')

Input: Look what you made me do .
Output: look what thou dost do me do .


In [0]:
output = predict('Father, please! Help me!')

Input: Father, please! Help me!
Output: father , i pray thee ! help me thee , i pray thee !


In [0]:
output = predict('What fight was here?')

Input: What fight was here?
Output: what fray was here ?


In [0]:
inp = 'i ’ ll bite you by the ear for that joke .'
output=predict(inp)

Input: i ’ ll bite you by the ear for that joke .
Output: i ll bite you by the ear of that jest .


In [0]:
output = predict('Good bye, Mr. Anderson.')

Input: Good bye, Mr. Anderson.
Output: farewell , good monsieur and law .


In [0]:
output = predict('what do you want ? ')

Input: what do you want ? 
Output: what would you ?


In [0]:
import nltk 
from nltk import bleu
import re
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4


def bleuscore(inp,predict):

  
  line = re.sub(r"[^A-Za-z\s]", "", inp.strip())
  words = line.split()
  inp=' '.join(words)

  line = re.sub(r"[^A-Za-z\s]", "", predict.strip())
  words = line.split()
  predict=' '.join(words)

  #print(inp,predict)

  ref=inp.split()
  hypo=predict.split()
  
  
  #score=nltk.translate.bleu_score.sentence_bleu([ref], hypo)
  #score=bleu([ref],hypo,smoothing_function=smoothie)
  score=bleu([ref],hypo)
  print(score)
  return score



In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
inputs=np.load('test.npy')

In [0]:
inputs.shape

(1000,)

In [0]:
inputs[0]

'For my own good, All other problems will have to wait.'

In [0]:
counter=1000
summ=0
for inp in inputs:

  output=predict(inp)
  x=bleuscore(inp,output)
  if x==0:
    counter-=1
  summ+=x

print(summ/counter)


Input: For my own good, All other problems will have to wait.
Output: for mine own good , all causes shall give way .
0.32689967585126034
Input: Does the king leave here today?
Output: doth the king leave to leave this morning ?
0.3655552228545123
Input: It’s just midnight.
Output: tis certain tis twelve .
0
Input: I cannot tell.
Output: i cannot tell .
0.7598356856515925
Input: O Dardanius!
Output: o dardanius !
0
Input: Good God, soon we’ll remove The thing that makes us strangers!
Output: good god , betimes remove the means that makes us strangers !
0.2513293635022765
Input: Get up, I say.
Output: arise , i say !
0.5444460596606694
Input: Wait!
Output: peace , ho !
0
Input: Honestly, then I’m afraid you are dammed both by father and mother; when I keep away from the Sea Monster, your father, I fall into an equal evil, your mother; well, you are gone both ways.
Output: by the sea i fear thou art sick and dusanines dug upon my father and liberty , which i keep thy father , if thy fath

**Reference:**
https://www.tensorflow.org/tutorials/text/transformer